In [6]:
import tiktoken
from huggingface_hub import hf_hub_download
import pandas as pd

dataset = pd.read_parquet(
    hf_hub_download(repo_id="openai/mrcr", filename="2needle.parquet", repo_type="dataset")
)


In [12]:
import json

enc = tiktoken.get_encoding("o200k_base")

def n_tokens(row) -> int:
    """
    Count tokens in messages.
    """
    messages = json.loads(row["prompt"])

    return sum([len(enc.encode(m["content"])) for m in messages])


In [10]:
MAX_CONTEXT_WINDOW= 64000

In [26]:
df = dataset[dataset.apply(lambda row: n_tokens(row) < MAX_CONTEXT_WINDOW, axis=1)]
len(df)

400

In [37]:
df2 = df[df.apply(lambda row: n_tokens(row) > (MAX_CONTEXT_WINDOW -30000), axis=1)]

In [38]:
len(df2)

94

In [40]:
df2.to_csv("MRCR-64000.csv")